In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from db_connection import connect_to_db
import pandas as pd
import pymysql
import warnings

In [2]:
credentials = connect_to_db()

In [3]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
engine = create_engine(f'mysql+pymysql://{credentials['user']}:{credentials['password']}@{credentials['host']}/{credentials['database']}')
sql_engine = engine.connect().connection
df = pd.read_sql('SELECT * FROM aviation_data;', sql_engine)

In [5]:
df.head()

,id,flight_date,flight_status,departure_airport,departure_timezone,departure_scheduled,departure_estimated,arrival_airport,arrival_timezone,arrival_scheduled,arrival_estimated,airline_name,flight_number,flight_icao
0,1,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,FlexFlight,4821,FXT4821
1,2,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,None,2821,A12821
2,3,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,None,4012,H14012
3,4,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 07:45:00,2025-03-08 07:45:00,Seoul (Incheon),Asia/Seoul,2025-03-08 10:35:00,2025-03-08 10:35:00,Virgin Atlantic,5530,VIR5530
4,5,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 07:45:00,2025-03-08 07:45:00,Seoul (Incheon),Asia/Seoul,2025-03-08 10:35:00,2025-03-08 10:35:00,Air France,5367,AFR5367


In [6]:
df.tail()

,id,flight_date,flight_status,departure_airport,departure_timezone,departure_scheduled,departure_estimated,arrival_airport,arrival_timezone,arrival_scheduled,arrival_estimated,airline_name,flight_number,flight_icao
815,816,2025-03-12,scheduled,Forest Hill,Australia/Sydney,2025-03-12 12:15:00,2025-03-12 12:15:00,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 13:25:00,NaT,Emirates,5179,UAE5179
816,817,2025-03-12,scheduled,Port Macquarie,Australia/Sydney,2025-03-12 12:25:00,2025-03-12 12:25:00,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 13:30:00,NaT,Air New Zealand,7087,ANZ7087
817,818,2025-03-12,scheduled,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 12:15:00,2025-03-12 12:15:00,Melbourne - Tullamarine Airport,Australia/Melbourne,2025-03-12 13:50:00,NaT,Qatar Airways,8486,QTR8486
818,819,2025-03-12,scheduled,Zhengzhou,Asia/Shanghai,2025-03-12 09:50:00,2025-03-12 09:50:00,Shenzhen,Asia/Shanghai,2025-03-12 12:30:00,NaT,Longhao Airlines,4031,LHA4031
819,820,2025-03-12,scheduled,Zhengzhou,Asia/Shanghai,2025-03-12 09:45:00,2025-03-12 09:45:00,Weihai,Asia/Shanghai,2025-03-12 11:40:00,NaT,Longhao Airlines,4011,LHA4011


In [7]:
df.nunique()

id                     820
flight_date              2
flight_status            4
departure_airport       64
departure_timezone      29
departure_scheduled     86
departure_estimated     86
arrival_airport         93
arrival_timezone        44
arrival_scheduled      122
arrival_estimated       64
airline_name           100
flight_number          349
flight_icao            356
dtype: int64

In [8]:
df.dtypes

id                              int64
flight_date                    object
flight_status                  object
departure_airport              object
departure_timezone             object
departure_scheduled    datetime64[ns]
departure_estimated    datetime64[ns]
arrival_airport                object
arrival_timezone               object
arrival_scheduled      datetime64[ns]
arrival_estimated      datetime64[ns]
airline_name                   object
flight_number                  object
flight_icao                    object
dtype: object

In [9]:
df['flight_status'].unique()

array(['scheduled', 'cancelled', 'active', 'landed'], dtype=object)

In [10]:
df['airline_name'].isnull().any().sum()

1

In [11]:
df['airline_name'].isna().any().sum()

1

In [12]:
def null_rows(df): 
    return df[df.isnull().any(axis=1)]

In [13]:
null_rows(df)

,id,flight_date,flight_status,departure_airport,departure_timezone,departure_scheduled,departure_estimated,arrival_airport,arrival_timezone,arrival_scheduled,arrival_estimated,airline_name,flight_number,flight_icao
1,2,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,None,2821,A12821
2,3,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,None,4012,H14012
21,22,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,None,2821,A12821
22,23,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,None,4012,H14012
49,50,2025-03-08,scheduled,Diosdado Macapagal International (Clark Intern...,Asia/Manila,2025-03-08 08:25:00,2025-03-08 08:25:00,Seoul (Incheon),Asia/Seoul,2025-03-08 13:50:00,2025-03-08 13:50:00,None,9767,H19767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,816,2025-03-12,scheduled,Forest Hill,Australia/Sydney,2025-03-12 12:15:00,2025-03-12 12:15:00,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 13:25:00,NaT,Emirates,5179,UAE5179
816,817,2025-03-12,scheduled,Port Macquarie,Australia/Sydney,2025-03-12 12:25:00,2025-03-12 12:25:00,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 13:30:00,NaT,Air New Zealand,7087,ANZ7087
817,818,2025-03-12,scheduled,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 12:15:00,2025-03-12 12:15:00,Melbourne - Tullamarine Airport,Australia/Melbourne,2025-03-12 13:50:00,NaT,Qatar Airways,8486,QTR8486
818,819,2025-03-12,scheduled,Zhengzhou,Asia/Shanghai,2025-03-12 09:50:00,2025-03-12 09:50:00,Shenzhen,Asia/Shanghai,2025-03-12 12:30:00,NaT,Longhao Airlines,4031,LHA4031


In [14]:
df["airline_name"].replace("empty", pd.NA, inplace=True)

In [15]:
# handling missing values
df['arrival_estimated'].fillna(df['arrival_scheduled'], inplace = True) 
df['departure_estimated'].fillna(df['departure_scheduled'], inplace = True) 
df['departure_timezone'].fillna(df['arrival_timezone'],inplace = True)
df["departure_airport"].fillna("Unknown Airport", inplace=True)
df["arrival_airport"].fillna("Unknown Airport", inplace=True)
df['arrival_timezone'].fillna("UTC", inplace=True)
df['airline_name'].fillna('Unknown', inplace = True)
df['flight_number'].fillna('Unknown', inplace=True)
df['flight_icao'].fillna('Unknown', inplace=True)

In [16]:
null_rows(df)

,id,flight_date,flight_status,departure_airport,departure_timezone,departure_scheduled,departure_estimated,arrival_airport,arrival_timezone,arrival_scheduled,arrival_estimated,airline_name,flight_number,flight_icao


In [17]:
# handling duplicate data
df.drop_duplicates(subset = df.columns.difference(['id']), keep = 'first')
df = df.reset_index(drop = True)

In [18]:
df

,id,flight_date,flight_status,departure_airport,departure_timezone,departure_scheduled,departure_estimated,arrival_airport,arrival_timezone,arrival_scheduled,arrival_estimated,airline_name,flight_number,flight_icao
0,1,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,FlexFlight,4821,FXT4821
1,2,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,Unknown,2821,A12821
2,3,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,Unknown,4012,H14012
3,4,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 07:45:00,2025-03-08 07:45:00,Seoul (Incheon),Asia/Seoul,2025-03-08 10:35:00,2025-03-08 10:35:00,Virgin Atlantic,5530,VIR5530
4,5,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 07:45:00,2025-03-08 07:45:00,Seoul (Incheon),Asia/Seoul,2025-03-08 10:35:00,2025-03-08 10:35:00,Air France,5367,AFR5367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,816,2025-03-12,scheduled,Forest Hill,Australia/Sydney,2025-03-12 12:15:00,2025-03-12 12:15:00,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 13:25:00,2025-03-12 13:25:00,Emirates,5179,UAE5179
816,817,2025-03-12,scheduled,Port Macquarie,Australia/Sydney,2025-03-12 12:25:00,2025-03-12 12:25:00,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 13:30:00,2025-03-12 13:30:00,Air New Zealand,7087,ANZ7087
817,818,2025-03-12,scheduled,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 12:15:00,2025-03-12 12:15:00,Melbourne - Tullamarine Airport,Australia/Melbourne,2025-03-12 13:50:00,2025-03-12 13:50:00,Qatar Airways,8486,QTR8486
818,819,2025-03-12,scheduled,Zhengzhou,Asia/Shanghai,2025-03-12 09:50:00,2025-03-12 09:50:00,Shenzhen,Asia/Shanghai,2025-03-12 12:30:00,2025-03-12 12:30:00,Longhao Airlines,4031,LHA4031


In [19]:
df = df.astype(str) 
df = df.drop_duplicates(subset=df.columns.difference(["id"]), keep="first")

In [20]:
df[df.duplicated(subset=df.columns.difference(["id"]), keep=False)]

,id,flight_date,flight_status,departure_airport,departure_timezone,departure_scheduled,departure_estimated,arrival_airport,arrival_timezone,arrival_scheduled,arrival_estimated,airline_name,flight_number,flight_icao


In [21]:
df

,id,flight_date,flight_status,departure_airport,departure_timezone,departure_scheduled,departure_estimated,arrival_airport,arrival_timezone,arrival_scheduled,arrival_estimated,airline_name,flight_number,flight_icao
0,1,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,FlexFlight,4821,FXT4821
1,2,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,Unknown,2821,A12821
2,3,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,Unknown,4012,H14012
3,4,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 07:45:00,2025-03-08 07:45:00,Seoul (Incheon),Asia/Seoul,2025-03-08 10:35:00,2025-03-08 10:35:00,Virgin Atlantic,5530,VIR5530
4,5,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 07:45:00,2025-03-08 07:45:00,Seoul (Incheon),Asia/Seoul,2025-03-08 10:35:00,2025-03-08 10:35:00,Air France,5367,AFR5367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,816,2025-03-12,scheduled,Forest Hill,Australia/Sydney,2025-03-12 12:15:00,2025-03-12 12:15:00,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 13:25:00,2025-03-12 13:25:00,Emirates,5179,UAE5179
816,817,2025-03-12,scheduled,Port Macquarie,Australia/Sydney,2025-03-12 12:25:00,2025-03-12 12:25:00,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 13:30:00,2025-03-12 13:30:00,Air New Zealand,7087,ANZ7087
817,818,2025-03-12,scheduled,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 12:15:00,2025-03-12 12:15:00,Melbourne - Tullamarine Airport,Australia/Melbourne,2025-03-12 13:50:00,2025-03-12 13:50:00,Qatar Airways,8486,QTR8486
818,819,2025-03-12,scheduled,Zhengzhou,Asia/Shanghai,2025-03-12 09:50:00,2025-03-12 09:50:00,Shenzhen,Asia/Shanghai,2025-03-12 12:30:00,2025-03-12 12:30:00,Longhao Airlines,4031,LHA4031


In [22]:
df = df.reset_index(drop = True)

In [23]:
df

,id,flight_date,flight_status,departure_airport,departure_timezone,departure_scheduled,departure_estimated,arrival_airport,arrival_timezone,arrival_scheduled,arrival_estimated,airline_name,flight_number,flight_icao
0,1,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,FlexFlight,4821,FXT4821
1,2,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,Unknown,2821,A12821
2,3,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 08:15:00,2025-03-08 08:15:00,Tan Son Nhat International,Asia/Ho_Chi_Minh,2025-03-08 13:05:00,2025-03-08 13:05:00,Unknown,4012,H14012
3,4,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 07:45:00,2025-03-08 07:45:00,Seoul (Incheon),Asia/Seoul,2025-03-08 10:35:00,2025-03-08 10:35:00,Virgin Atlantic,5530,VIR5530
4,5,2025-03-08,scheduled,Haneda Airport,Asia/Tokyo,2025-03-08 07:45:00,2025-03-08 07:45:00,Seoul (Incheon),Asia/Seoul,2025-03-08 10:35:00,2025-03-08 10:35:00,Air France,5367,AFR5367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,816,2025-03-12,scheduled,Forest Hill,Australia/Sydney,2025-03-12 12:15:00,2025-03-12 12:15:00,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 13:25:00,2025-03-12 13:25:00,Emirates,5179,UAE5179
362,817,2025-03-12,scheduled,Port Macquarie,Australia/Sydney,2025-03-12 12:25:00,2025-03-12 12:25:00,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 13:30:00,2025-03-12 13:30:00,Air New Zealand,7087,ANZ7087
363,818,2025-03-12,scheduled,Sydney Kingsford Smith Airport,Australia/Sydney,2025-03-12 12:15:00,2025-03-12 12:15:00,Melbourne - Tullamarine Airport,Australia/Melbourne,2025-03-12 13:50:00,2025-03-12 13:50:00,Qatar Airways,8486,QTR8486
364,819,2025-03-12,scheduled,Zhengzhou,Asia/Shanghai,2025-03-12 09:50:00,2025-03-12 09:50:00,Shenzhen,Asia/Shanghai,2025-03-12 12:30:00,2025-03-12 12:30:00,Longhao Airlines,4031,LHA4031


In [24]:
df.nunique()

id                     366
flight_date              2
flight_status            4
departure_airport       65
departure_timezone      32
departure_scheduled     86
departure_estimated     86
arrival_airport         94
arrival_timezone        45
arrival_scheduled      122
arrival_estimated      122
airline_name           100
flight_number          350
flight_icao            357
dtype: int64

In [25]:
df.dtypes

id                     object
flight_date            object
flight_status          object
departure_airport      object
departure_timezone     object
departure_scheduled    object
departure_estimated    object
arrival_airport        object
arrival_timezone       object
arrival_scheduled      object
arrival_estimated      object
airline_name           object
flight_number          object
flight_icao            object
dtype: object

In [32]:
# reverting to original data type
df['id'] = df['id'].astype('int64')
df['flight_date'] = pd.to_datetime(df['flight_date'], errors='coerce')
df['departure_scheduled'] = pd.to_datetime(df['departure_scheduled'], errors='coerce')
df['departure_estimated'] = pd.to_datetime(df['departure_estimated'], errors='coerce')
df['arrival_scheduled'] = pd.to_datetime(df['arrival_scheduled'], errors='coerce')
df['arrival_estimated'] = pd.to_datetime(df['arrival_estimated'], errors='coerce')

In [33]:
df.dtypes

id                              int64
flight_date            datetime64[ns]
flight_status                  object
departure_airport              object
departure_timezone             object
departure_scheduled    datetime64[ns]
departure_estimated    datetime64[ns]
arrival_airport                object
arrival_timezone               object
arrival_scheduled      datetime64[ns]
arrival_estimated      datetime64[ns]
airline_name                   object
flight_number                  object
flight_icao                    object
dtype: object